### Polynomial hash: Deep dive

- In many algorithms, there is a need to efficiently compare the differences/similarity between 2 strings
    - Rabin Karp is one such case, where we want to efficiently find the occurrences of substrings 

- We have explored in the chapter notes how we can use hashing to accomplish this. In this extra set of notes, we will deep dive into why the polynomial hash works

- Let's first implement a polynomial hash

- Why use polyhash?
    - The logic of polynomial hash is covered pretty well in the lecture; the idea here is that with polyhash, we can compute the difference between overlapping strings in constant time (subtract the first character * some polynomial, add the last character * some polynomial)

- Why use modulo?
    - Modulo is added to prevent overflow, since the size of the integer grows exponentially

- Why must the polynomial $p$ chosen be larger than the size of the character set $|\Sigma|$>
    - Intuitively, the smaller your polynomial, the greater the chance of collisions
    - Why? Because the span of the values is simply smaller. Let's take an extreme case; imagine the polynomial is 2:
        - Hash value of $2^2 = \{\text{a, ba}\}$
        - So many strings can give you the same hash value, which means that collisions are more likely!

- A basic implementation is provided below

In [23]:
polynomial = 31
prime = 1e9 + 7

def polynomial_hash_basic(string):
    hashval = 0
    for i, char in enumerate(string):
        print(f'Hashing {char=} with {ord(char)=}; {i=}, {polynomial**i=}, {hashval=}')
        hashval +=  ord(char) * (polynomial**i)
    return hashval

(polynomial_hash_basic('some longer string')) > (2**64)

Hashing char='s' with ord(char)=115; i=0, polynomial**i=1
Hashing char='o' with ord(char)=111; i=1, polynomial**i=31
Hashing char='m' with ord(char)=109; i=2, polynomial**i=961
Hashing char='e' with ord(char)=101; i=3, polynomial**i=29791
Hashing char=' ' with ord(char)=32; i=4, polynomial**i=923521
Hashing char='l' with ord(char)=108; i=5, polynomial**i=28629151
Hashing char='o' with ord(char)=111; i=6, polynomial**i=887503681
Hashing char='n' with ord(char)=110; i=7, polynomial**i=27512614111
Hashing char='g' with ord(char)=103; i=8, polynomial**i=852891037441
Hashing char='e' with ord(char)=101; i=9, polynomial**i=26439622160671
Hashing char='r' with ord(char)=114; i=10, polynomial**i=819628286980801
Hashing char=' ' with ord(char)=32; i=11, polynomial**i=25408476896404831
Hashing char='s' with ord(char)=115; i=12, polynomial**i=787662783788549761
Hashing char='t' with ord(char)=116; i=13, polynomial**i=24417546297445042591
Hashing char='r' with ord(char)=114; i=14, polynomial**i=75

True

- The basic implementation runs into several issues immediately;
    - `polynomial ** i` can easily overflow the 64 bit integer boundary
    - Even if the polynomial is ok, the sum of the polynomial hash can exceed the boundary rapidly, especially if the character size is large

- To overcome this, we add a modulo at 2 points
    - Whenever we add each term, we take modulo of the current hashval 
    - Whenever we multiply the polynomial, we take modulo of the current polynomial value

In [33]:
polynomial = 31
prime = 1e9 + 7

def polynomial_hash(string):
    hashval = 0
    p = 1
    for i, char in enumerate(string):
        # print(f'Hashing {char=} with {ord(char)=}; {i=}, {p=}, {hashval=}')
        hashval = (hashval + (ord(char) * p)) % prime
        p = (p * polynomial) % prime
    return hashval

polynomial_hash('some long string')

213920304.0

### Applying polynomial hashing in substring hash

- Let's assume we have following string $s = \text{coding}$, and the substring $p = \text{din}$
    - Applying the polynomial hash to $s$, we end up with the expression $c \cdot x^0 + o \cdot x^1 + d \cdot x^2 + i \cdot x^3 + n \cdot x^4 + g \cdot x^5$
    - Applying the polynomial hash to $p$, we end up with the expression $d \cdot x^0 + i \cdot x^1 + n \cdot x^2$

- For the same string `din` in $s$ and $p$, it contributes a different amount to the eventual hash. BUT there is an obvious realtionship in the contribution to both hashes for the matching pattern:
    - Contribution of `din` in $s$ is $H_s(\text{din}) = d \cdot x^2 + i \cdot x^3 + n \cdot x^4$
    - Contribution of `din` in $p$ is $H_p(\text{din}) = d \cdot x^0 + i \cdot x^1 + n \cdot x^2$
    - So $H_p(\text{din}) = \frac{H_s(\text{din})}{x^2}$

- The idea here is that the hash of a substring can be easily computed in $O(1)$ time, by taking the contribution of that substring, divided by the polynomial raised to the index of the first character

### Prefix array

- One common idea here is to use a dynamic programming solution to pre-compute a prefix array for a given string
    - This will help you get to the hash value of a given substring in O(1) time!

- Reusing the example above where $s = \text{coding}$
    - The prefix array $h$ will simply be $[c \cdot x^0, \quad c \cdot x^0 + o \cdot x^1, \quad c \cdot x^0 + o \cdot x^1 + d \cdot x^2, ...]$

- Let's suppose we have $h$, and we want to compute the value of some substring of $s$. Let's call this substring $s[i : i+j]$ where j is the length of the substring
    - We can compute this in O(1) time using $h$ by taking $\frac{h[i+j-1] - h[i-1]}{x^i}$
    - Let's assume we want to get the hash of `din` in $s$. This is $s[2:5]$
    - $h[2+3-1] = h[4]$ is the polynomial hash of `codin`, and $h[2-1] = h[1]$ is the polynomial hash of `co`. Hence, the difference between the two values is the contribution of the substring to the original hash
    - To find the value of this standalone substring, we simply divide it by the polynomial raised to index $i$

In [15]:
hashval = [3,3,13,251,]
(hashval[-1] if hashval else 0) + 10283

10534

In [32]:
polynomial = 31
prime = 1e9 + 7
string = 'trololol'
window_size = 3

def polynomial_hash(s: str, return_prefix_hash: bool=False) -> list[int] | int:
    if return_prefix_hash:
        hashval = []
        p = [1]
        for i, char in enumerate(string):
            # print(f'Hashing {char=} with {ord(char)=}; {i=}, {p=}, {hashval=}')
            hashval_new = ((hashval[-1] if hashval else 0) + (ord(char) * p[-1])) % prime
            p_new = (p[-1] * polynomial) % prime
            hashval.append(hashval_new)
            p.append(p_new)
        return hashval
    else:
        hashval = 0
        p = 1
        for i, char in enumerate(string):
            # print(f'Hashing {char=} with {ord(char)=}; {i=}, {p=}, {hashval=}')
            hashval = (hashval + (ord(char) * p)) % prime
            p = (p * polynomial) % prime
        return hashval

prefix_hash = polynomial_hash(s=string, return_prefix_hash=True)

# def get_substr_hash(first_index: int, last_index: int):
first_index = 2
last_index = 4

if first_index == 0:
    print(prefix_hash[last_index])
else:
    temp = (prefix_hash[last_index] - prefix_hash[first_index-1])/(polynomial**2)
    temp = temp + prime if temp < 0 else temp
    temp = (temp * pow(int(polynomial), int(len(string)-first_index), int(prime))) % prime
    print(temp)

polynomial_hash(s=string[first_index:(last_index+1)])

779704350.0


72997956.0

In [44]:
class RollingHash:
    def __init__(self, s):
        self.length = len(s)
        self.mod1 = 10**9 + 7
        self.mod2 = 10**9 + 9
        self.p1 = 31
        self.p2 = 37
        self.hash1 = [0] * self.length
        self.hash2 = [0] * self.length
         
        # Compute hashes of the string s
         
        h1 = h2 = 0
        p_pow1 = p_pow2 = 1
        for i in range(self.length):
            h1 = (h1 + (ord(s[i]) - ord('a') + 1) * p_pow1) % self.mod1
            h2 = (h2 + (ord(s[i]) - ord('a') + 1) * p_pow2) % self.mod2
            p_pow1 = (p_pow1 * self.p1) % self.mod1
            p_pow2 = (p_pow2 * self.p2) % self.mod2
            self.hash1[i] = h1
            self.hash2[i] = h2
 
    # Returns the hash value of the prefix of s up to index i
    def prefix(self, index):
        return (self.hash1[index], self.hash2[index])
     
    # # Returns the hash value of the substring of s from index l to r (inclusive)
    # def substr(self, l, r):
    #     if l == 0:
    #         return (self.hash1[r], self.hash2[r])
    #     temp1 = self.hash1[r] - self.hash1[l-1]
    #     temp2 = self.hash2[r] - self.hash2[l-1]
    #     temp1 += self.mod1 if temp1 < 0 else 0
    #     temp2 += self.mod2 if temp2 < 0 else 0
    #     temp1 = (temp1 * pow(self.p1, self.length-l, self.mod1)) % self.mod1
    #     temp2 = (temp2 * pow(self.p2, self.length-l, self.mod2)) % self.mod2
    #     return (temp1, temp2)
    def substr(self, l, r):
        if l == 0:
            return (self.hash1[r], self.hash2[r])
        temp1 = (self.hash1[r] - self.hash1[l-1] + self.mod1) % self.mod1
        temp2 = (self.hash2[r] - self.hash2[l-1] + self.mod2) % self.mod2
        temp1 = (temp1 * pow(self.p1, self.length - l, self.mod1)) % self.mod1
        temp2 = (temp2 * pow(self.p2, self.length - l, self.mod2)) % self.mod2
        return (temp1, temp2)
 
    def __eq__(self, other):
        return self.prefix(self.length-1) == other.prefix(other.length-1)
 
 
my_str = "geeksforgeeks"
left = 2
right = 5
print(my_str[left:(right+1)])
hash = RollingHash(my_str)
print(hash.substr(2,5))

hashcheck = RollingHash(my_str[left:(right+1)])
hashcheck.hash1

eksf
(581069439, 152602678)


[5, 346, 18605, 197351]

#### Extra: Euclidean Algorithm

- Euclidean algorithm
    - Let's assume there are 2 integers $A, B$ that have a common divisor $d$
        - In notation, we write that $d | A$ and $d | B$ (i.e. d divides A and B)
    - If $d | A$ and $d | B$, then it must be true that $d | (x \cdot A + y \cdot B)$
        - Any linear combination of A and B must be divisible by $d$. Think of A and B as a product of some sequence of divisors, including $d$. Then multiplying an additional $x$ to that sequence does not change that fact that it must still be divisible by $d$
    - We can always represent $A$ as $A = q \cdot B + r$, assuming $A > B$. 
        - If $A < B$ just flip the numbers.
        - $q$ is some arbitary integer, and $r$ is the remainder
    - But this can be rewritten as $r = A - q \cdot B$, which is precisely a linear combination of A and B
        - Therefore, $r$ must also be divisible by $d$

    - Let $d$ be a common divisor of $A$ and $B$.
        - $d | A$, $d | B$
        - By the argument above, it must be true that, $d | A - qB$
        - Therefore, $d | r$
    - Let $e$ be a common divisor of $B$ and $r$
        - $e | B$, $e | r$
        - By the argument above, it must be true that, $e | qB + r$
        - Therefore, $e | A$

    - So every divisor of A and B must also be a divisor for $r$, and every divisor for B and r must also be a divisor for A
        - Therefore, since the set of divisors $d \equiv e$, then $gcd(A,B) \equiv gcd(B,r)$
 

In [49]:
def gcd(a, b):
    if (a == 0):
        return b
    elif (b == 0):
        return a
    else:
        ## Tail recursion
        return gcd(b%a, a)

- From the above discussion, given $d | A$ and $d | B$, then $d | (xA + yB)$
- Beyond returning the GCD, the extended Euclidean algorithm also returns the values of $x, y$ such that $gcd(A,B) = xA + yB$

$$\begin{aligned}
    ax_1 + by_1 &= \text{gcd}(a,b) \\
    \text{gcd}(a,b) &= \text{gcd}(b \mod a,a) \\
    \text{gcd}(b \mod a,a) &= b \mod a x_2 + a y_2 \\ \\

    \therefore ax_1 + by_1 &= b \mod a x_2 + a y_2 \\
    ax_1 + by_1 &= (b - \left \lfloor  \frac{b}{a} \right \rfloor \cdot a) x_2 + a y_2 \\
    &= b x_2 - \left \lfloor  \frac{b}{a} \right \rfloor \cdot a \cdot x_2 + a y_2 \\
    &= a [y_2 - \left \lfloor  \frac{b}{a} \right \rfloor \cdot x_2] + b x_2 \\ \\

    \text{Comparing coefficients...} \\
    \therefore x_1 &= y_2 - \left \lfloor  \frac{b}{a} \right \rfloor \cdot x_2 \\
    \therefore y_1 &= x_2  


\end{aligned}$$

In [56]:
def gcd_extended(a, b):
    if a == 0:
        x1 = 0
        y1 = 1
        return b, x1, y1
    
    gcd, x2, y2 = gcd_extended(b%a, a)
    x1 = y2 - (b//a) * x2
    y1 = x2

    return gcd, x1, y1
    
gcd_extended(35, 15)

(5, 1, -2)

#### Application: Computing Inverse Modulo

- In modulo arithmetic, there is no division operation!! 
    - Instead of dividing, we multiply it by the modular inverse of the denominator!
    - For revision of modular arithmetic: https://brilliant.org/wiki/modular-arithmetic/

- The modular inverse of of $A \mod M$ is defined as the integer $B$ such that $(A \cdot B) \equiv 1 \mod M$
    - Naive approach: Try all values between $1$ and $M-1$. $O(M)$ time complexity
    - Better approach: Use extended Euclidean algorithm! (see section below)
    - What's the idea here?
        - To take the modular inverse of $A \mod M$, we are asking for $B$ such that $(A \cdot B) \mod M \equiv 1 \mod M = 1$
        - Let's first observe that this $B$ only exists when $A$ and $M$ are coprime
            - If they are not, then no value $B$ can make $A \cdot B \equiv 1 \mod M$
            - Because if $gcd(A, M) > 1$, then $gcd(AB, M) > 1$, since the product of 1 extra term to A does not change the original gcd value
        - From the extended Euclidean algorithm, we get `gcd, x, y` such that $gcd(A,M) = xA + yM$
            - From modular arithemtic, this means that $gcd(A,M) \mod M = xA \mod M + yM \mod M$
            - We already know that $gcd(A,M) = 1$, and by definition $yM \mod M = 0$
        - Rewriting, $A \cdot x \mod M = 1$
        - But this is exactly what we're trying to find! So $x = B$ is a valid answer

In [6]:
def gcd_extended(a, b):
    if a == 0:
        x1 = 0
        y1 = 1
        return b, x1, y1
    
    gcd, x2, y2 = gcd_extended(b%a, a)
    x1 = y2 - (b//a) * x2
    y1 = x2

    return gcd, x1, y1

def inverse_mod(a, m):
    gcd, x, y = gcd_extended(a, m)
    
    ## If the gcd of a and m is larger than 1, then there is no way for the gcd of a*b to be 1, because the original larger divisor is still there
    if gcd != 1:
        return None  
    else:
        # return x % m
        return x